In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas  as pd
import pickle
import os
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [4]:
os.listdir('../../')

['TransE',
 'data.zip',
 '.git',
 'f2w.nt',
 'PTransE',
 'TransH',
 'Trans_PY',
 'CTransR',
 'wikidata',
 'FB15k',
 'README.md',
 'makefile',
 'cluster',
 '__MACOSX',
 'WN18']

In [5]:
data_path = '../../wikidata/'

## Relation Information analysis

In [6]:
with open(data_path+'train.txt', 'r') as f:
    data = f.readlines()
    
tail2relation_map = {}
head2relation_map = {}
entity_list = []

for i in data:
    head, tail, rel = i.split('\t')
    head = head.strip()
    tail = tail.strip()
    rel = rel.strip()
    if head not in entity_list:
        entity_list += [head]
    if tail not in entity_list:
        entity_list += [tail]
    if head not in head2relation_map:
        head2relation_map.update({head : [rel]})
    elif rel not in head2relation_map[head]:
        head2relation_map[head] += [rel]
    
    if tail not in tail2relation_map:
        tail2relation_map.update({tail : [rel]})
    elif rel not in tail2relation_map[tail]:
        tail2relation_map[tail] += [rel]

In [7]:
def get_str(lis):
    st = ''
    for i in lis:
        st += i.strip() + '\t'
    return st.strip()

In [8]:
one2one_relation = {}
one2multi_relation = {}
multi2one_relation = {}
multi2multi_relation = {}
false_entity = []
one2one_cnt = 0
one2multi_cnt = 0
multi2one_cnt = 0
multi2multi_cnt = 0

for i in entity_list:
    if i not in tail2relation_map or i not in head2relation_map:
        if i not in false_entity:
            false_entity += [i]
        continue
    if len(tail2relation_map[i]) > 1 and len(head2relation_map[i]) > 1:
        multi2multi_cnt += 1
        if get_str(tail2relation_map[i]) not in multi2multi_relation:
            multi2multi_relation.update({get_str(tail2relation_map[i]):[get_str(head2relation_map[i])]})
        elif get_str(head2relation_map[i]) not in multi2multi_relation[get_str(tail2relation_map[i])]:
            multi2multi_relation[get_str(tail2relation_map[i])] += [get_str(head2relation_map[i])]
    if len(tail2relation_map[i]) == 1 and len(head2relation_map[i]) > 1:
        one2multi_cnt += 1
        if tail2relation_map[i][0] not in one2multi_relation:
            one2multi_relation.update({tail2relation_map[i][0]:[get_str(head2relation_map[i])]})
        elif get_str(head2relation_map[i]) not in one2multi_relation[tail2relation_map[i][0]]:
            one2multi_relation[tail2relation_map[i][0]] += [get_str(head2relation_map[i])]
    if len(tail2relation_map[i]) > 1 and len(head2relation_map[i]) == 1:
        multi2one_cnt += 1
        if get_str(tail2relation_map[i]) not in multi2one_relation:
            multi2one_relation.update({get_str(tail2relation_map[i]):[head2relation_map[i][0]]})
        elif head2relation_map[i][0] not in multi2one_relation[get_str(tail2relation_map[i])]:
            multi2one_relation[get_str(tail2relation_map[i])] += [head2relation_map[i][0]]
    if len(tail2relation_map[i]) == 1 and len(head2relation_map[i]) == 1:
        one2one_cnt += 1
        if get_str(tail2relation_map[i]) not in one2one_relation:
            one2one_relation.update({get_str(tail2relation_map[i]):[get_str(head2relation_map[i])]})
        elif get_str(head2relation_map[i]) not in one2one_relation[get_str(tail2relation_map[i])]:
            one2one_relation[get_str(tail2relation_map[i])] += [get_str(head2relation_map[i])]

In [9]:
multi2multi_cnt, multi2one_cnt, one2multi_cnt, one2one_cnt

(12829, 190, 268, 628)

In [10]:
len(multi2multi_relation), len(multi2one_relation), len(one2multi_relation), len(one2one_relation)

(11543, 145, 85, 149)

In [21]:
total_pairs = []
rel_list = []
def get_pairs(dict1):
    lis_pairs = []
    global rel_list, total_pairs
    for keys,values in dict1.items():
        lis = keys.split('\t')
        for i in lis:
            if i not in rel_list:
                rel_list += [i]
            for j in lis:
                if i != j and i + '\t' + j not in lis_pairs:
                    lis_pairs += [i+'\t'+j]
                if i != j and i + '\t' + j not in total_pairs:
                    total_pairs += [i+'\t'+j]
        for rels in values:
            r_lis = rels.split('\t')
            for i in r_lis:
                if i not in rel_list:
                    rel_list += [i]
                for j in r_lis:
                    if i != j and i + '\t' + j not in lis_pairs:
                        lis_pairs += [i+'\t'+j]
                    if i != j and i + '\t' + j not in total_pairs:
                        total_pairs += [i+'\t'+j]
    return lis_pairs
# one2one_pairs = get_pairs(one2one_relation)
one2multi_pairs = get_pairs(one2multi_relation)
multi2one_pairs = get_pairs(multi2one_relation)

In [22]:
len(rel_list), len(one2one_pairs), len(multi2one_pairs), len(one2multi_pairs)

(372, 160, 1330, 934)

In [33]:
# len(one2one_pairs),one2one_pairs[0]
# multi2one_pairs

In [29]:
with open(data_path+'relation_graph.txt', 'w') as f:
    for i in total_pairs:
        f.write(i+'\n')

In [14]:
# df = pd.DataFrame(list(multi2multi_relation.values()), index = list(multi2multi_relation.keys()))
# pd.set_option('max_columns', 10000000)
# df.head()
# df['first'] = df.index
# df.index = range(len(df))
# df.columns = ['first', 'additive1', 'additive2', 'second']
# df[['first', 'second', 'additive1', 'additive2']]
# with open(data_path+'multi2multi_relation.csv', 'w') as f:
#     df.to_csv(f)

In [44]:
str1 = '/location/statistical_region/rent50_2./measurement_unit/dated_money_value/currency'
dict1 = {}
for i in total_pairs:
    head, tail = i.split('\t')
#     print(head, tail)
    if str1 == head:
        if tail not in dict1:
            dict1.update({tail:1})
        else:
            dict1[tail] += 1
    if tail == str1:
        if head not in dict1:
            dict1.update({head:1})
        else:
            dict1[head] += 1

In [45]:
pd.DataFrame(list(dict1.values()), index=list(dict1.keys()))

,0
/location/administrative_division/second_level_division_of,2
/location/location/time_zones,2
/location/statistical_region/rent50_0./measurement_unit/dated_money_value/currency,2
/location/statistical_region/rent50_1./measurement_unit/dated_money_value/currency,2
/location/hud_foreclosure_area/total_90_day_vacant_residential_addresses./measurement_unit/dated_integer/source,2
/location/statistical_region/rent50_4./measurement_unit/dated_money_value/currency,2
/location/hud_foreclosure_area/total_residential_addresses./measurement_unit/dated_integer/source,2
/location/hud_foreclosure_area/hhuniv./measurement_unit/dated_integer/source,2
/location/hud_foreclosure_area/estimated_number_of_mortgages./measurement_unit/dated_integer/source,2
/location/location/containedby,2


In [111]:
def search_wikidata(index):
    url = "https://www.wikidata.org/wiki/"+index
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, "html.parser")
#     print(index)
    label = ''
    desc = ''
    for tag in soup.find_all("span", class_="wikibase-title-label"):
        label = tag.string
    for tag in soup.find_all("span", class_="wikibase-descriptionview-text"):	
        desc = tag.string
    return label + '-->' + desc

In [175]:
with open(data_path+'train.txt', 'r') as f:
    data = f.readlines()
    
relation2head_map = {}
relation2tail_map = {}
rel_h2t = {}
rel_t2h = {}
for i in data:
    head, tail, rel = i.split('\t')
    head = head.strip()
    tail = tail.strip()
    rel = rel.strip()
    if rel + '\t' + head not in rel_h2t:
        rel_h2t.update({rel + '\t' + head : [tail]})
    elif tail not in rel_h2t[rel + '\t' + head]:
        rel_h2t[rel + '\t' + head] += [tail]
    
    if rel + '\t' + tail not in rel_t2h:
        rel_t2h.update({rel + '\t' + tail : [head]})
    elif head not in rel_t2h[rel + '\t' + tail]:
        rel_t2h[rel + '\t' + tail] += [head]
        
    if rel not in relation2head_map:
        relation2head_map.update({rel : [head]})
    elif head not in relation2head_map[rel]:
        relation2head_map[rel] += [head]
    
    if rel not in relation2tail_map:
        relation2tail_map.update({rel : [tail]})
    elif tail not in relation2tail_map[rel]:
        relation2tail_map[rel] += [tail]

## one2one relation analysis

In [179]:
relation_pairs_entity = {}
for rel1, values in one2one_relation.items():
    for rel2 in values:
        for e in relation2tail_map[rel1]:
            if  rel2+'\t'+e in rel_h2t:
                relation_pairs_entity.update({rel1 + '\t' + rel2 : [rel_t2h[rel1+'\t'+e][0],
                                                                   e, rel_h2t[rel2+'\t'+e][0]]})
                break

In [200]:
relation_pairs_desc = {}
for key, values in relation_pairs_entity.items():
    if key not in relation_pairs_desc:
        relation_pairs_desc.update({key: [search_wikidata(values[0])+'\n' +
                                           search_wikidata(values[1])+'\n' + search_wikidata(values[2])]})
    else:
        relation_pairs_desc[key] += [search_wikidata(values[0])+'\n' + 
                                      search_wikidata(values[1])+'\n' + search_wikidata(values[2])]

In [182]:
df = pd.DataFrame(list(relation_pairs_desc.values()), index=list(relation_pairs_desc.keys()))
with open(data_path+'one2one_relation_desc.csv', 'w') as f:
    df.to_csv(f)

## Relation sim analysis

In [20]:
with open(data_path+'train.txt', 'r') as f:
    data = f.readlines()

relation_entity_map = {}
for i in data:
    head, tail, rel = i.split('\t')
    if rel not in relation_entity_map:
        relation_entity_map.update({rel:{'head':[head],'tail':[tail]}})
    else:
        if head not in relation_entity_map[rel]['head']:
            relation_entity_map[rel]['head'] += [head]
        if tail not in relation_entity_map[rel]['tail']:
            relation_entity_map[rel]['tail'] += [tail]

In [48]:
relation_sim_pairs = []
for key,values in relation_entity_map.items():
    for k, val in relation_entity_map.items():
        head_cnt = 0
        tail_cnt = 0
        if k != key:
            for head in values['head']:
                if head in val['head']:
                    head_cnt += 1
                    if head_cnt > 100:
                        break;
            for tail in values['tail']:
                if tail in val['tail']:
                    tail_cnt += 1
                    if tail_cnt > 100:
                        break
        if head_cnt > 100 and tail_cnt > 100:
            relation_sim_pairs += [key+'\t'+k]

In [49]:
df = pd.DataFrame(relation_sim_pairs)
df.columns = ['triple']
df['first'] = df['triple'].apply(lambda x: x.split('\t')[0].strip())
df['second'] = df['triple'].apply(lambda x: x.split('\t')[1].strip())

In [51]:
pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 1000)
# df[['first','second']]

## Relation relation analysis

In [183]:
with open(data_path+'train.txt', 'r') as f:
    data = f.readlines()
    
relation2head_map = {}
relation2tail_map = {}
head2relation_map = {}
relation_list = []

for i in data:
    head, tail, rel = i.split('\t')
    if rel not in relation_list:
        relation_list += [rel]
    if head in head2relation_map and rel in head2relation_map[head]:
        head2relation_map[head][rel] += [tail]
    elif head in head2relation_map:
        head2relation_map[head].update({rel:[tail]})
    else:
        head2relation_map[head] = {rel:[tail]}
    if rel in relation2head_map and head in relation2head_map[rel]:
        if tail in relation2head_map[rel][head]:
            print(head, tail, rel)
        relation2head_map[rel][head] += [tail]
    elif rel in relation2head_map:
        relation2head_map[rel].update({head:[tail]})
    else:
        relation2head_map[rel] = {head:[tail]}
    
    if rel in relation2tail_map and tail in relation2tail_map[rel]:
        if head in relation2tail_map[rel][tail]:
            print(head, tail, rel)
        relation2tail_map[rel][tail] += [head]
    elif rel in relation2tail_map:
        relation2tail_map[rel].update({tail:[head]})
    else:
        relation2tail_map[rel] = {tail:[head]}

In [184]:
relation_pairs_entity_map = {}
for key, values in relation2tail_map.items():
    lis = []
    for tail,_ in values.items():
        if tail in head2relation_map:
            for i in head2relation_map[tail].keys():
                if key + '\t' + i in relation_pairs_entity_map:
#                     if tail not in relation_pairs_entity_map[key+'\t'+i]:
                    relation_pairs_entity_map[key+'\t'+i] += [tail]
                else:
                    relation_pairs_entity_map.update({key+'\t'+i:[tail]})

In [187]:
relation_pairs_entity_cnt = {}
for key, values in relation_pairs_entity_map.items():
    if len(values) < 1000:
        relation_pairs_entity_cnt[key] = len(values)

In [188]:
# plt.bar(range(len(relation_pairs_entity_cnt)), relation_pairs_entity_cnt.values())
len(relation_pairs_entity_cnt)

50966

In [191]:
reduced_rel_list = []
for i in relation_pairs_entity_cnt.keys():
    r1, r2 = i.split('\t')
    if r1.strip() not in reduced_rel_list:
        reduced_rel_list += [r1.strip()]
    if r2.strip() not in reduced_rel_list:
        reduced_rel_list += [r2.strip()]

In [192]:
len(reduced_rel_list)

1291

In [193]:
reduced_triples = []
for i in data:
    head, tail, rel = i.split("\t")
    if rel.strip() in reduced_rel_list:
        reduced_triples += [i]

In [194]:
len(reduced_triples)

421198

In [195]:
len(data)

421198

In [196]:
len(relation_pairs_entity_cnt)

50966

In [197]:
pd.set_option('max_columns', 1000)
df = pd.DataFrame(list(relation_pairs_entity_cnt.values()), index = list(relation_pairs_entity_cnt.keys())).sort_values(
 [0], ascending=[False])

In [198]:
df['first'] = df.index
df.index = range(len(df))
df.columns = ['cnt', 'triple']
df['first'] = df['triple'].apply(lambda x: x.split('\t')[0].strip())
df['second'] = df['triple'].apply(lambda x: x.split('\t')[1].strip())

In [199]:
with open(data_path+'relation_pairs_small.csv', 'w') as f:
    df[['first', 'second', 'cnt']].to_csv(f)

## FB convert to WikiData

In [ ]:
src_path = '../../FB15k/'
dest_path = '../../wikidata/'

fb2wiki = {}
with open(src_path+'f2w.nt', 'r') as f:
    data = f.readlines()
    for i in data:
        fb, wiki = i.split(' ')
        fb2wiki[fb] = wiki.rstrip()

with open(src_path+'test.txt', 'r') as f:
    data = f.readlines()
triples = []
for i in data:
    h, t, rel = i.split('\t')
    if h in fb2wiki and t in fb2wiki:
        triples += [fb2wiki[h]+'\t'+fb2wiki[t]+'\t'+rel]

with open(dest_path+'test.txt', 'w+') as f:
    for i in triples:
        f.write(i)

os.listdir(dest_path)